In [66]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import librosa
import os 

In [67]:
# Ajuster les options d'affichage
pd.set_option('display.max_columns', None)  # Aucune limite sur le nombre de colonnes affichées
pd.set_option('display.max_colwidth', None)  # Aucune limite sur la largeur du contenu de la colonne

In [68]:
path_audible_5s_test_csv = "../../Dataset3/CSVs/Audible/test_audible_recording_chunks.csv"
path_audible_5s_train_csv = "../../Dataset3/CSVs/Audible/train_audible_recording_chunks.csv"
path_audible_5s_val_csv = "../../Dataset3/CSVs/Audible/val_audible_recording_chunks.csv"

# Using pandas to read the CSV file into a DataFrame
data_test = pd.read_csv(path_audible_5s_test_csv)
data_train = pd.read_csv(path_audible_5s_train_csv)
data_val = pd.read_csv(path_audible_5s_val_csv)

C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\2334637947.py:7: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  data_train = pd.read_csv(path_audible_5s_train_csv)


In [69]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data_train['label'])

def clean_data(df):
    labels = df[['label','code_unique','chunk_initial_time','chunk_final_time','sampling_rate','label_class']]

    labels["label_str"] = labels["label"]

    labels['label'] = label_encoder.transform(labels['label'])

    return labels

#data_val = clean_data(data_val)
data_train = clean_data(data_train)

num_labels = max(data_train['label']) + 1
print(num_labels)

173


C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\1382958805.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["label_str"] = labels["label"]
C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\1382958805.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['label'] = label_encoder.transform(labels['label'])


In [70]:
label_encoder = LabelEncoder()
label_encoder.fit(data_test['label'])

data_test = clean_data(data_test)

C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\1382958805.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["label_str"] = labels["label"]
C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\1382958805.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['label'] = label_encoder.transform(labels['label'])


In [71]:
liste_labels = []
liste_labels2 = []

for i in range(num_labels):
    label_str = data_train[data_train["label"] == i]["label_str"].values[0]
    liste_labels.append(label_str)

for i in range(120):
    label_str = data_test[data_test["label"] == i]["label_str"].values[0]
    liste_labels2.append(label_str)
    if label_str not in liste_labels:
        print(label_str)

print(liste_labels)
print(liste_labels2)


Hippolais polyglotta
['Aegithalos caudatus', 'Alarme', 'Alauda arvensis', 'Alytes obstetricans', 'Anthus trivialis', 'Apis mellifera', 'Avion', 'Bicolorana bicolor', 'Bombina variegata', 'Bos taurus', 'Branche', 'Bug informatique', 'Buteo buteo', 'Camion', 'Canis familiaris', 'Certhia brachydactyla', 'Certhia familiaris', 'Chanson', 'Choc', 'Chorthippus apricarius', 'Chorthippus biguttulus', 'Chorthippus brunneus', 'Chorthippus dorsatus', 'Chorthippus jacobsi', 'Chorthippus mollis', 'Chorthippus saulcyi', 'Chorthippus vagans', 'Chrysochraon dispar', 'Cicada orni', 'Cicadatra atra', 'Cicadetta petryi', 'Cisticola juncidis', 'Cloche', 'Cloche_v', 'Columba palumbus', 'Conocephalus fuscus', 'Corvus corax', 'Corvus corone', 'Cris', 'Curruca communis', 'Curruca melanocephala', 'Cyanistes caeruleus', 'Decticus albifrons', 'Decticus verrucivorus', 'Delichon urbicum', 'Dendrocopos major', 'Disqueuse', 'Déplacement', 'Déplacement animal', 'Eau', 'Emberiza calandra', 'Emberiza cia', 'Emberiza cir

In [72]:
path_predictions_csv = "BirdNET-Analyzer-main/example/test2/"

files_pred = os.listdir(path_predictions_csv)

data_pred_list = []

for file in files_pred:
    data_pred = []
    with open(path_predictions_csv + file, 'r') as file:
        # Skip the header line if it exists
        header = file.readline()

        # Loop through each line in the file
        for line in file:
            # Split the line into individual values using tab as the delimiter
            values = line.strip().split('\t')

            # Create a dictionary using column names as keys
            data_dict = {
                'Selection': int(values[0]),
                'View': values[1],
                'Channel': int(values[2]),
                'Begin Path': values[3],
                'File Duration (s)': float(values[4]),
                'Begin Time (s)': float(values[5]),
                'End Time (s)': float(values[6]),
                'Low Freq (Hz)': float(values[7]),
                'High Freq (Hz)': float(values[8]),
                'Species Code': values[9],
                'Common Name': values[10],
                'Confidence': float(values[11]),
            }

            # Append the dictionary to the list
            data_pred.append(data_dict)

    data_pred_list.append(data_pred)

print(len(data_pred_list))

889


In [74]:
path_audible_5s_test_audio = "../../Dataset3/Selection morceaux audio 5s/Audible/test/"
files_test = os.listdir(path_audible_5s_test_audio)

y_test = []
for file in files_test:
    split_str_file = file.split("_split_")
    split_2 = split_str_file[1].split("_")
    chunk_initial_time = split_2[0]
    code = split_str_file[0]
    condition1 = data_test["code_unique"]==code
    condition2 = data_test["chunk_initial_time"]==int(chunk_initial_time)
    labels = data_test[condition1 & condition2]["label_str"].values
    labels_categorical = [0] * num_labels
    for label_str in labels:
        if label_str != "Hippolais polyglotta":
            idx = liste_labels.index(label_str)
            labels_categorical[idx] = 1
    y_test.append(labels_categorical)

In [75]:
y_test[3]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [76]:
liste_labels = []

for i in range(num_labels):
    label_str = data_train[data_train["label"] == i]["label_str"].values[0]
    liste_labels.append(label_str)

liste_labels = [label.replace("é","e") for label in liste_labels]
liste_labels = [label.replace("ç","c") for label in liste_labels]
print(liste_labels)

['Aegithalos caudatus', 'Alarme', 'Alauda arvensis', 'Alytes obstetricans', 'Anthus trivialis', 'Apis mellifera', 'Avion', 'Bicolorana bicolor', 'Bombina variegata', 'Bos taurus', 'Branche', 'Bug informatique', 'Buteo buteo', 'Camion', 'Canis familiaris', 'Certhia brachydactyla', 'Certhia familiaris', 'Chanson', 'Choc', 'Chorthippus apricarius', 'Chorthippus biguttulus', 'Chorthippus brunneus', 'Chorthippus dorsatus', 'Chorthippus jacobsi', 'Chorthippus mollis', 'Chorthippus saulcyi', 'Chorthippus vagans', 'Chrysochraon dispar', 'Cicada orni', 'Cicadatra atra', 'Cicadetta petryi', 'Cisticola juncidis', 'Cloche', 'Cloche_v', 'Columba palumbus', 'Conocephalus fuscus', 'Corvus corax', 'Corvus corone', 'Cris', 'Curruca communis', 'Curruca melanocephala', 'Cyanistes caeruleus', 'Decticus albifrons', 'Decticus verrucivorus', 'Delichon urbicum', 'Dendrocopos major', 'Disqueuse', 'Deplacement', 'Deplacement animal', 'Eau', 'Emberiza calandra', 'Emberiza cia', 'Emberiza cirlus', 'Emberiza citri

In [77]:
liste_labels.index("Voix")

171

In [78]:
y_pred = []

treshold_confidence = 0.5

for pred in data_pred_list:
    pred_categorical = [0]*num_labels
    for selec in pred:
        if selec['Species Code'] == "nocall":
            pass
        elif selec['Confidence'] > treshold_confidence:
            predicted_label = selec['Species Code'].replace("Ã©","e").replace("Ã§","c")
            predicted_label = liste_labels.index(predicted_label)
            pred_categorical[predicted_label] = 1
    y_pred.append(pred_categorical)

In [86]:
is_insecte = [0]*num_labels

for i in range(num_labels):
    label_class_str = data_train[data_train["label"] == i]["label_class"].values[0]
    is_insecte[i] = (label_class_str=="Insecta")
is_insecte[liste_labels.index("Mouche")] = False
print(is_insecte)
num_labels_insecta =sum(is_insecte)

print(num_labels_insecta)

[False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, True, False, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, False, True, True, True, True, False, False, False, False, False, False, True, True, False, False, True, True, True, False, False, False, False, True, False, False, True, False, True, False, True, True, False, False, False, False, False, False, True, True, False, True, True, True, True, False, False, False, False, False, False, False, False, True, True, True, True, False, False, False, False, True, True, True, False, False, True, False, True, True, True, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, True, True, True, True, False, False, False

In [87]:
y_pred_insecta = []
y_truth_insecta = []

for i in range(len(y_pred)):
    new_pred = []
    new_truth = []
    for j in range(num_labels):    
        if is_insecte[j]:
            new_pred.append(y_pred[i][j])
            new_truth.append(y_test[i][j])
    y_pred_insecta.append(new_pred)
    y_truth_insecta.append(new_truth)

In [88]:
liste_labels_insectes = []

for i in range(num_labels):
    if is_insecte[i]:
        liste_labels_insectes.append(liste_labels[i])

print(liste_labels_insectes)

['Apis mellifera', 'Bicolorana bicolor', 'Chorthippus apricarius', 'Chorthippus biguttulus', 'Chorthippus brunneus', 'Chorthippus dorsatus', 'Chorthippus jacobsi', 'Chorthippus mollis', 'Chorthippus saulcyi', 'Chorthippus vagans', 'Chrysochraon dispar', 'Cicada orni', 'Cicadatra atra', 'Cicadetta petryi', 'Conocephalus fuscus', 'Decticus albifrons', 'Decticus verrucivorus', 'Ephippiger diurnus', 'Ephippiger terrestris', 'Euchorthippus declivus', 'Euchorthippus elegantulus', 'Eumodicogryllus bordigalensis', 'Euthystira brachyptera', 'Gomphocerippus rufus', 'Gomphocerus sibiricus', 'Gryllotalpa gryllotalpa', 'Gryllus bimaculatus', 'Gryllus campestris', 'Leptophyes punctatissima', 'Lyristes plebejus', 'Melanogryllus desertus', 'Metrioptera buyssoni', 'Metrioptera saussuriana', 'Myrmeleotettix maculatus', 'Nemobius sylvestris', 'Oecanthus pellucens', 'Omocestus raymondi', 'Omocestus rufipes', 'Omocestus viridulus', 'Phaneroptera laticerca', 'Phaneroptera nana', 'Pholidoptera femorata', 'Ph

In [89]:
num_correct = 0

true_positives = [0]*num_labels_insecta
false_positives = [0]*num_labels_insecta
false_negatives = [0]*num_labels_insecta
true_negatives = [0]*num_labels_insecta

i = 0
for predicted_l in y_pred_insecta:
    predicted = np.array(predicted_l)
    truth = np.array(y_truth_insecta[i])

    true_positives += ((predicted == 1) & (truth == 1))
    false_positives += ((predicted == 1) & (truth == 0))
    false_negatives += ((predicted == 0) & (truth == 1))
    true_negatives += ((predicted == 0) & (truth == 0))

    num_correct += np.all(truth-predicted == 0)

    i += 1

print(num_correct/i)

0.49718785151856015


In [90]:
accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)
precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
F1 = 2 * precision * recall / (precision + recall)

accuracy = np.nan_to_num(accuracy)
precision = np.nan_to_num(precision)
recall = np.nan_to_num(recall)
F1 = np.nan_to_num(F1)

print("accuracy \t precision \t recall \t F1 \t classe")
for i in range(len(accuracy)):
    print(f'{accuracy[i]:.2f} \t \t {precision[i]:.2f} \t \t {recall[i]:.2f} \t \t {F1[i]:.2f} \t {liste_labels_insectes[i]}')

print('\n')

accuracy_mean = round(np.nanmean(accuracy),3)
precision_mean =round(np.nanmean(precision),3)
recall_mean = round(np.nanmean(recall),3)
F1_mean = round(np.nanmean(F1),3)

print("Moyennes:\t" + str(accuracy_mean) + "\t" + str(precision_mean) + "\t" + str(recall_mean) + "\t" + str(F1_mean))

accuracy 	 precision 	 recall 	 F1 	 classe
1.00 	 	 1.00 	 	 0.79 	 	 0.88 	 Apis mellifera
1.00 	 	 0.83 	 	 0.62 	 	 0.71 	 Bicolorana bicolor
1.00 	 	 0.60 	 	 0.75 	 	 0.67 	 Chorthippus apricarius
1.00 	 	 1.00 	 	 0.83 	 	 0.91 	 Chorthippus biguttulus
0.99 	 	 0.50 	 	 0.11 	 	 0.18 	 Chorthippus brunneus
0.99 	 	 0.00 	 	 0.00 	 	 0.00 	 Chorthippus dorsatus
0.99 	 	 1.00 	 	 0.42 	 	 0.59 	 Chorthippus jacobsi
0.99 	 	 0.64 	 	 0.75 	 	 0.69 	 Chorthippus mollis
0.99 	 	 0.56 	 	 0.38 	 	 0.45 	 Chorthippus saulcyi
0.99 	 	 0.67 	 	 0.60 	 	 0.63 	 Chorthippus vagans
0.99 	 	 0.67 	 	 0.18 	 	 0.29 	 Chrysochraon dispar
0.99 	 	 0.90 	 	 0.82 	 	 0.86 	 Cicada orni
1.00 	 	 1.00 	 	 1.00 	 	 1.00 	 Cicadatra atra
0.99 	 	 1.00 	 	 0.45 	 	 0.62 	 Cicadetta petryi
0.98 	 	 0.00 	 	 0.00 	 	 0.00 	 Conocephalus fuscus
1.00 	 	 1.00 	 	 0.73 	 	 0.84 	 Decticus albifrons
0.97 	 	 1.00 	 	 0.66 	 	 0.79 	 Decticus verrucivorus
0.98 	 	 0.39 	 	 0.50 	 	 0.44 	 Ephippiger diurnus


C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\2750595247.py:2: RuntimeWarning: invalid value encountered in divide
  precision = true_positives / (true_positives + false_positives)
C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\2750595247.py:3: RuntimeWarning: invalid value encountered in divide
  recall = true_positives / (true_positives + false_negatives)
C:\Users\cdelm\AppData\Local\Temp\ipykernel_183424\2750595247.py:4: RuntimeWarning: invalid value encountered in divide
  F1 = 2 * precision * recall / (precision + recall)
